# 네이버 지식인 크롤링
- keyword 임의 설정

In [1]:
import time
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from urllib.parse import quote
from tqdm import tqdm
from datetime import datetime, timedelta

## 날짜 입력
- 며칠동안의 데이터를 수집할지 입력
- 입력값 N = 3 -> 3일전 ~ 오늘날짜

In [2]:
KEYWORD = '전기차'
N = 5
start_date = datetime.today() - timedelta(N)
FROM_DATE = start_date.strftime("%Y.%m.%d.")
TO_DATE = datetime.today().strftime("%Y.%m.%d.")

In [25]:
print(FROM_DATE, TO_DATE)

2021.04.15. 2021.04.20.


In [27]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
base_url = "https://www.naver.com/"
driver.get(base_url)
time.sleep(1)

# 지식iN 으로 이동
nav = driver.find_element_by_id('NM_FAVORITE')
lis = nav.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
lis[3].click()

# 키워드 검색 후 최신순 정렬
search_area = driver.find_element_by_class_name('search_area')
search_area.find_element_by_tag_name('input').send_keys(KEYWORD)    # 키워드 입력
driver.find_element_by_class_name('search_btn').click()     # 검색
driver.find_element_by_class_name('option_sort').find_elements_by_tag_name('li')[1].click()     # '최신순' 클릭
time.sleep(1)

# 기간 검색: FROM_DATE ~ TO_DATE
driver.find_element_by_id('au_date_select').click()
time.sleep(1)
from_date = driver.find_element_by_id('sel_from_date')
to_date = driver.find_element_by_id('sel_to_date')
# 시작날짜, 완료날짜 입력
actions = ActionChains(driver).double_click(from_date).send_keys(FROM_DATE).double_click(to_date).send_keys(TO_DATE)
actions.perform()
driver.find_element_by_xpath('/html/body/div[3]/div/ul/li[4]/input[3]').click()

# 전체 페이지수 수집
while True:
    pagination = driver.find_element_by_class_name('s_paging')
    pages = pagination.find_elements_by_tag_name('a')
    tmp = []
    for page in pages:
        tmp.append(page.text)
    if '다음페이지' in tmp:
        pages[-1].click()
        time.sleep(2)
    else:
        NUM_OF_PAGE = int(pagination.text.split()[-1])
        break
print("전체 페이지 수: ", NUM_OF_PAGE)
driver.close()

전체 페이지 수:  20


In [28]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
ans_url_list = []; total_ans = 0

# 질문들의 url 수집
for page in range(NUM_OF_PAGE):
    kin_url = f'https://kin.naver.com/search/list.nhn?sort=date&query={quote(KEYWORD)}'
    period_url = f'&period={FROM_DATE}%7C{TO_DATE}&section=kin'
    page_url = f'&page={page+1}'
    url = kin_url + period_url + page_url
    driver.get(url)
    time.sleep(1)
    #print(f"{page+1} 페이지")

    ul = driver.find_element_by_xpath('//*[@id="s_content"]/div[3]/ul')
    lis = ul.find_elements_by_tag_name('li')
    for li in lis:
        a_tag = li.find_element_by_tag_name('a')
        ans_url_list.append(a_tag.get_attribute('href'))

print(f'총 {len(ans_url_list)}개의 질문')

# 각 질문의 답변 개수
for i, ans_url in enumerate(ans_url_list):
    driver.get(ans_url)
    time.sleep(0.5)
    ans_cnt = int(driver.find_element_by_class_name('_answerCount.num').text)
    total_ans += ans_cnt
    print(f'{i+1}번째 : {ans_cnt}')
print("----------------------------------")
print(f'Total 답변 개수: {total_ans}')

driver.close()

총 196개의 질문
1번째 : 2
2번째 : 1
3번째 : 3
4번째 : 7
5번째 : 1
6번째 : 2
7번째 : 1
8번째 : 1
9번째 : 1
10번째 : 4
11번째 : 2
12번째 : 11
13번째 : 3
14번째 : 3
15번째 : 1
16번째 : 5
17번째 : 5
18번째 : 1
19번째 : 7
20번째 : 15
21번째 : 1
22번째 : 1
23번째 : 1
24번째 : 3
25번째 : 3
26번째 : 1
27번째 : 5
28번째 : 7
29번째 : 15
30번째 : 2
31번째 : 10
32번째 : 3
33번째 : 2
34번째 : 3
35번째 : 13
36번째 : 1
37번째 : 1
38번째 : 2
39번째 : 15
40번째 : 6
41번째 : 2
42번째 : 1
43번째 : 1
44번째 : 1
45번째 : 7
46번째 : 20
47번째 : 1
48번째 : 1
49번째 : 7
50번째 : 1
51번째 : 13
52번째 : 5
53번째 : 2
54번째 : 1
55번째 : 2
56번째 : 1
57번째 : 1
58번째 : 2
59번째 : 3
60번째 : 2
61번째 : 1
62번째 : 6
63번째 : 1
64번째 : 1
65번째 : 1
66번째 : 2
67번째 : 1
68번째 : 6
69번째 : 6
70번째 : 2
71번째 : 5
72번째 : 2
73번째 : 9
74번째 : 1
75번째 : 1
76번째 : 5
77번째 : 4
78번째 : 1
79번째 : 3
80번째 : 1
81번째 : 4
82번째 : 5
83번째 : 11
84번째 : 1
85번째 : 1
86번째 : 4
87번째 : 5
88번째 : 1
89번째 : 1
90번째 : 4
91번째 : 1
92번째 : 6
93번째 : 3
94번째 : 3
95번째 : 26
96번째 : 5
97번째 : 9
98번째 : 18
99번째 : 7
100번째 : 8
101번째 : 1
102번째 : 3
103번째 : 6
104번째 : 2
105번째 : 8
106번째 : 8
107번째 : 3
108번째 : 2
109번째 

In [29]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
answer_text_list = []

for ans_url in tqdm(ans_url_list):
    driver.get(ans_url)
    time.sleep(1)
    more = driver.find_element_by_xpath('//*[@id="nextPageButton"]')
    disp = more.get_attribute('style')      # 더보기 할 답변이 남아있지 않은 경우 "style:None" 이 존재한다
    try:
        # 더보기 할 답변이 남지 않을 때까지 while문을 돌린다
        while disp:     
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(1)
    except:
        time.sleep(1)
    answers = driver.find_elements_by_class_name('answer-content__item._contentWrap._answer')
    for ans in answers:
        ps = ans.find_elements_by_tag_name('p')
        for p in ps:
            answer_text_list.append(p.text)
driver.close()

100%|██████████| 196/196 [11:47<00:00,  3.61s/it]


In [30]:
len(answer_text_list)

15368

### 원본 저장(전처리 X)

In [31]:
df = pd.DataFrame({
    'answer': answer_text_list
})
df.head()

,answer
0,wher**** 님 답변
1,
2,현대위아 주가를 비롯해 전기차 관련주들이 가끔 한번씩 이슈를 타곤 하지만 현대위아 ...
3,
4,


In [3]:
start = start_date.strftime("%y%m%d")
end = datetime.today().strftime("%y%m%d")
print(start, end)

210415 210420


In [33]:
df.to_csv(f'./data/{KEYWORD}_{start}_to_{end}_org.txt', sep='\t', index=False)

### 데이터 전처리

In [4]:
df_test = pd.read_table(f'./data/{KEYWORD}_{start}_to_{end}_org.txt')
df_test.head()

,answer
0,wher**** 님 답변
1,NaN
2,현대위아 주가를 비롯해 전기차 관련주들이 가끔 한번씩 이슈를 타곤 하지만 현대위아 ...
3,NaN
4,NaN


In [5]:
df_test.shape

(14752, 1)

In [6]:
df_test['answer'] = df_test['answer'].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df_test.head()

<ipython-input-6-e7bed6f86f98>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['answer'] = df_test['answer'].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,answer
0,wher 님 답변
1,NaN
2,현대위아 주가를 비롯해 전기차 관련주들이 가끔 한번씩 이슈를 타곤 하지만 현대위아 ...
3,NaN
4,NaN


In [7]:
# 공백 여부 확인
drop_idx = df_test[df_test.answer == ""].index
print(len(drop_idx), drop_idx)

569 Int64Index([   94,   353,   748,  1015,  1096,  1105,  1142,  1166,  1170,
             1192,
            ...
            14459, 14586, 14591, 14632, 14643, 14652, 14667, 14669, 14698,
            14751],
           dtype='int64', length=569)


In [9]:
# 공백 값을 가지고 있는 해당 인덱스 행 제거
df_test.drop(drop_idx, inplace=True)
df_test.shape

(14183, 1)

In [10]:
# null값 여부 확인
df_test.isnull().sum()

answer    3999
dtype: int64

In [11]:
# null값을 가지고 있는 행 제거
df_test.dropna(axis=0, inplace=True)
df_test.shape

(10184, 1)

In [12]:
df_test.head()

,answer
0,wher 님 답변
2,현대위아 주가를 비롯해 전기차 관련주들이 가끔 한번씩 이슈를 타곤 하지만 현대위아 ...
5,위 답변은 답변작성자가 경험과 지식을 바탕으로 작성한 내용입니다 포인트로 감사할 때...
6,시간 전
7,비공개 답변


In [13]:
print(f'전처리 후 샘플의 개수: {len(df_test)}')

전처리 후 샘플의 개수: 10184


In [14]:
df_test.to_csv(f'./result/{KEYWORD}_{start}_to_{end}_res.txt', sep='\t', index=False)